## Closed Loop Simulation Notebook

This notebook breaks down the closed loop simulation in different jupyter notebook cells. The aim of this notebook is to ease the debugging and development of the different packages involved in the Optical Feedback Control loop.

This notebook was last run on July 7th 2022. 
Details:
- Stack release: 2021_25, Last verified to run 2022-07-07
- ts_phosim release: v2.0.4


### Required python packages

In [1]:
import argparse
import logging
import numpy as np
import os
import shutil
import subprocess
import sys
import time
import batoid

from lsst.ts.batoid.CloseLoopTask import CloseLoopTask
from lsst.ts.batoid.telescope.TeleFacade import TeleFacade
from lsst.ts.batoid.utils.PlotUtil import plotFwhmOfIters
from lsst.daf import butler as dafButler
from lsst.ts.wep.Utility import CamType, FilterType, runProgram
from lsst.ts.batoid.utils.Utility import getPhoSimPath, getAoclcOutputPath, getCamera
from lsst.ts.batoid.utils.SensorWavefrontError import SensorWavefrontError
from lsst.ts.batoid.wfsim.wfsim import SSTBuilder

from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt

In [2]:
%%time

# Set the parser
parser = argparse.ArgumentParser(
    description="Run AOS closed-loop simulation (default is amplifier files)."
)
parser = CloseLoopTask.setDefaultParser(parser)
parser = CloseLoopTask.setImgParser(parser)

# Get the default arguments
sys.argv = ['-f']
args = parser.parse_args()

# Print default arguments
print(args)

logger = logging.getLogger()
logger.setLevel(args.log_level)

# Initialize the ClosedLoopTask
closeLoopTask = CloseLoopTask()

Namespace(inst='comcam', filterType='', rotCam=0.0, m1m3FErr=0.05, numOfProc=1, iterNum=5, output='', log_level=20, clobber=False, pipelineFile='', boresightDeg=[0, 0], skyFile='', eimage=False)
CPU times: user 3.77 ms, sys: 1.46 ms, total: 5.23 ms
Wall time: 4.13 ms


In [3]:
PHOSIMPATH = "/home/gmegias/jhome/aos/phosim_syseng4/"
AOCLCOUTPUTPATH = "/home/gmegias/jhome/aos/ts_batoid/output/"
os.environ["PHOSIMPATH"] = PHOSIMPATH
os.environ["AOCLCOUTPUTPATH"] = AOCLCOUTPUTPATH

args.inst = 'comcam' 
args.boresightDeg = [0.03, -0.02]
args.eimage = None
args.skyFile = '/home/gmegias/jhome/aos/ts_batoid/tests/testData/sky/skyComCam.txt'
args.output = '/home/gmegias/jhome/aos/perturbations/test2253/'

if os.path.exists(args.output):
    shutil.rmtree(args.output)

In [4]:
boresight = args.boresightDeg
rotCamInDeg = args.rotCam
useEimg = args.eimage
m1m3ForceError = args.m1m3FErr
numPro = args.numOfProc
iterNum = args.iterNum
doErsDirCont = args.clobber
pathSkyFile = args.skyFile
args.wavelength = 500e-9
args.numOfZk = 19

In [5]:
# Check the input arguments
camType, instName = closeLoopTask.getCamTypeAndInstName(args.inst)

closeLoopTask.configOfcCalc(instName)
closeLoopTask.configPhosimCmpt(
    instName, args.wavelength, args.filterType, rotCamInDeg, m1m3ForceError, numPro, boresight=boresight
)

In [6]:
# Set the telescope state to be the same as the OFC
dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state

# Get the list of referenced sensor name (field positions)

# If using wavefront sensors we measure one per pair
# and the field
if camType == CamType.LsstCam:
    cornerSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
    cornerSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)
    refSensorNameList = []
    refSensorIdList = []
    for name, id in zip(cornerSensorNameList, cornerSensorIdList):
        if name.endswith("SW0"):
            refSensorNameList.append(name)
            refSensorIdList.append(id)
else:
    refSensorNameList = closeLoopTask.getSensorNameListOfFields(instName)
    refSensorLocationList = closeLoopTask.getSensorLocationListOfFields(instName)
    refSensorIdList = closeLoopTask.getSensorIdListOfFields(instName)


iterCount = 0
obsId = 9006000
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)
# The iteration directory
iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
closeLoopTask.phosimCmpt.setOutputDir(outputDir)

outputDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
closeLoopTask.phosimCmpt.setOutputDir(outputDir)


In [7]:
zenith_angle = 27.0912
rotation_angle = 0.0
builder = SSTBuilder(batoid.Optic.fromYaml("ComCam_g.yaml"))

if True:
    builder = (
        builder
        .with_m1m3_gravity(np.deg2rad(zenith_angle))
        .with_m1m3_temperature(
            m1m3_TBulk=0.0902,
            m1m3_TxGrad=-0.0894,
            m1m3_TyGrad=-0.1973,
            m1m3_TzGrad=-0.0316,
            m1m3_TrGrad=0.0187
        )
        .with_m1m3_lut(np.deg2rad(zenith_angle), error=0.0, seed = 6)
    )
if True:
    builder = (
        builder
        .with_m2_gravity(np.deg2rad(zenith_angle))
        .with_m2_temperature(
            m2_TrGrad=-0.1416,
            m2_TzGrad=-0.0675
        )
    )
if True:
    builder = (
        builder
        .with_camera_gravity(
            zenith_angle=np.deg2rad(zenith_angle),
            rotation_angle=np.deg2rad(rotation_angle)
        )
        .with_camera_temperature(
            camera_TBulk=6.5650
        )
    )
optic = builder.build()

In [22]:
builder = builder.with_aos_dof(dofInUm)
optic = builder.build()

iterCount += 1
outputDirName = "pert"
outputImgDirName = "img"
iterDefaultDirName = "iter"
baseOutputDir = closeLoopTask.checkAndCreateBaseOutputDir(args.output)
# The iteration directory
iterDirName = "%s%d" % (iterDefaultDirName, iterCount)

outputDir = os.path.join(baseOutputDir, iterDirName, outputDirName)
closeLoopTask.phosimCmpt.setOutputDir(outputDir)

outputDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)
closeLoopTask.phosimCmpt.setOutputDir(outputDir)

In [23]:

def runBatoid(optic, wavelength, numOfZk, sensorIdList, sensorLocationList, outputDir, iterCount, obsId):

    listOfWfErr = []
    for sensorId, sensorLocation in zip(sensorIdList, sensorLocationList):

        zk = batoid.zernike(
            optic,
            sensorLocation[0], sensorLocation[1],
            wavelength, 
            eps=0.61, 
            jmax = numOfZk + 3, 
            nx=25
        ) * 0.5

        sensorWavefrontData = SensorWavefrontError(numOfZk=numOfZk)
        sensorWavefrontData.setSensorId(sensorId)
        #Batoid returns null zk[0] which is unused
        sensorWavefrontData.setAnnularZernikePoly(zk[4:])

        listOfWfErr.append(sensorWavefrontData)

        opd = batoid.wavefront(
            optic,
            sensorLocation[0], sensorLocation[1],
            wavelength=wavelength, 
            nx=255
        ).array * wavelength * 1e6
        
        opddata = opd.data.astype(np.float32)
        opddata[opd.mask] = 0.0
        ofn = os.path.join(outputDir, f"opd_{obsId}_{sensorId}.fits.gz")
        fits.writeto(
            ofn,
            opddata,
            overwrite=True
        )

    return listOfWfErr


listOfWfErr = runBatoid(optic, args.wavelength, args.numOfZk, refSensorIdList, refSensorLocationList, outputDir, iterCount, obsId)


In [24]:
rotOpdInDeg = 0
opdZkFileName = "opd.zer"
opdPssnFileName = "PSSN.txt"
outputDirName = "pert"

outputImgDirName = "img"
outputImgDir = os.path.join(baseOutputDir, iterDirName, outputImgDirName)

closeLoopTask.phosimCmpt.metr.setWgtAndFieldXyOfGQ(instName)
closeLoopTask.phosimCmpt.setOutputImgDir(outputImgDir)

closeLoopTask.phosimCmpt._writeOpdZkFile(opdZkFileName, rotOpdInDeg)
closeLoopTask.phosimCmpt._writeOpdPssnFile(instName, opdPssnFileName)

/home/gmegias/jhome/aos/perturbations/test2253/iter3/img
file list
['/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_0.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_1.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_2.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_3.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_4.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_5.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_6.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_7.fits.gz', '/home/gmegias/jhome/aos/perturbations/test2253/iter3/img/opd_9006030_8.fits.gz']
0.5
pssn list
[0.9630100958716726, 0.9653007345819246, 0.9628216070364727, 0.9649590026442961, 0.9655336593305265, 0.9646383533232652, 0.9607933331929144, 0.9635650804989531, 0.9604421380514069]

In [25]:
# Get the PSSN from file
pssn = closeLoopTask.phosimCmpt.getOpdPssnFromFile(opdPssnFileName)
print("Calculated PSSN is %s." % pssn)

# Get the GQ effective FWHM from file
gqEffFwhm = closeLoopTask.phosimCmpt.getOpdGqEffFwhmFromFile(opdPssnFileName)
print("GQ effective FWHM is %.4f." % gqEffFwhm)

# Set the FWHM data
print("Setting FWHM data...")
t0 = time.time()
fwhm, sensor_id = closeLoopTask.phosimCmpt.getListOfFwhmSensorData(
    opdPssnFileName, refSensorIdList
)

closeLoopTask.ofcCalc.set_fwhm_data(fwhm, sensor_id)
ttl_time = time.time() - t0
print("FWHM data set, took {} seconds.".format(ttl_time))

Calculated PSSN is [0.9630101  0.96530073 0.96282161 0.964959   0.96553366 0.96463835
 0.96079333 0.96356508 0.96044214].
GQ effective FWHM is 0.1269.
Setting FWHM data...
FWHM data set, took 0.0017521381378173828 seconds.


In [26]:
# Record the wavefront error with the same order as OPD for the
# comparison


# Calculate the DOF
print('Calculating DOF...')
t0 = time.time()
wfe = np.array(
    [sensor_wfe.getAnnularZernikePoly() for sensor_wfe in listOfWfErr]
)
sensor_ids = np.array(
    [sensor_wfe.getSensorId() for sensor_wfe in listOfWfErr]
)
ttl_time = time.time() - t0
print('DOF computed. Took {} seconds.'.format(ttl_time))

print('Calculating corrections...')
t0 = time.time()
closeLoopTask.ofcCalc.calculate_corrections(
    wfe=wfe,
    field_idx=sensor_ids,
    filter_name=str('R'),
    gain=-1,
    rot=rotCamInDeg,
)
ttl_time = time.time() - t0
print('Corrections computed. Took {} seconds.'.format(ttl_time))

# Set the new aggregated DOF to phosimCmpt
print('Setting and saving new DOF...')
t0 = time.time()
dofInUm = closeLoopTask.ofcCalc.ofc_controller.aggregated_state


# Save the DOF file
closeLoopTask.phosimCmpt.saveDofInUmFileForNextIter(
    dofInUm
)
ttl_time = time.time() - t0
print('New DOF saved. Took {} seconds.'.format(ttl_time))

# Add the observation ID by 10 for the next iteration
obsId += 10

Calculating DOF...
DOF computed. Took 0.000888824462890625 seconds.
Calculating corrections...
Corrections computed. Took 0.013141155242919922 seconds.
Setting and saving new DOF...
New DOF saved. Took 0.0037169456481933594 seconds.


The wavefront error of the first simulation iteration is printed below

### Summarize the FWHM and plot results

In [29]:
# Summarize the FWHM
fwhmItersFileName = "fwhmIters.png"
pssnFiles = [
    os.path.join(
        baseOutputDir,
        "%s%d" % (iterDefaultDirName, num),
        outputImgDirName,
        opdPssnFileName,
    )
    for num in [0,1,2,3]
]
saveToFilePath = os.path.join(baseOutputDir, fwhmItersFileName)
plotFwhmOfIters(pssnFiles, saveToFilePath=saveToFilePath)

In [30]:
# importing Image class from PIL package
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

im = np.array(Image.open(saveToFilePath))
plt.figure(figsize=(10,10))
plt.imshow(im)
plt.axis('off')
plt.show()


/tmp/ipykernel_1804689/2572018048.py:10: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [ ]:
for idx in [0]:

    image_file = get_pkg_data_filename('/home/gmegias/jhome/aos/perturbations/test2253/iter0/img/opd_9006000_4.fits.gz')
    image_data = fits.getdata(image_file, ext=0)
    masked =np.ma.masked_where(image_data == 0, image_data)

    image_file = get_pkg_data_filename('/home/gmegias/jhome/aos/perturbations/testNe3/iter0/img/opd_9006000_4.fits.gz')
    image_data2 = fits.getdata(image_file, ext=0)
    masked2 =np.ma.masked_where(image_data2 == 0, image_data2)


    print(np.min(image_data))
    print(np.max(image_data))

    plt.figure(figsize = (10,9))
    plt.imshow(abs(masked - masked2))
    plt.colorbar()
    plt.savefig('/home/gmegias/jhome/aos/perturbations/trial/diff_imagsse0{}s.png'.format(idx))
    plt.close()